In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import pymorphy2

In [2]:
df = pd.read_csv('test_data.csv')

In [3]:
df.dlg_id.value_counts()

5    143
0    109
2     85
1     55
3     53
4     35
Name: dlg_id, dtype: int64

In [4]:
df

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


### Реплики с приветствием менеджера

In [5]:
def extract_greeting(dataframe)->list[tuple()]:
    greeting_str = re.compile('здравствуйте|добрый день')
    greeting_indeces = []
    for idx, text in enumerate(dataframe.text):
        if dataframe.role[idx] == 'client':
            continue
        if greeting_str.findall(text.lower()):
            greeting_indeces.append((dataframe.dlg_id[idx], dataframe.line_n[idx]))
    return greeting_indeces

In [6]:
# test greeting
greeting_indeces = extract_greeting(df)
for dlg, line in greeting_indeces:
    print('dlg_id -',dlg, 'line_n -', line, ':', 
          df[(df.dlg_id == dlg) & (df.line_n == line)].text.iloc[0])

dlg_id - 0 line_n - 1 : Алло здравствуйте
dlg_id - 1 line_n - 1 : Алло здравствуйте
dlg_id - 2 line_n - 2 : Алло здравствуйте
dlg_id - 3 line_n - 1 : Алло дмитрий добрый день


### Реплики с прощанием менеджера

In [7]:
def extract_goodbye(dataframe)->list[tuple()]:
    goodbye_str = re.compile('всего хорошего|до свидания|всего доброго')
    goodbye_indeces = []
    for idx, text in enumerate(dataframe.text):
        if dataframe.role[idx] == 'client':
            continue
        if goodbye_str.findall(text.lower()):
            goodbye_indeces.append((dataframe.dlg_id[idx], dataframe.line_n[idx]))
    return goodbye_indeces

In [8]:
# test goodbye
goodbye_indeces = extract_goodbye(df)
for dlg, line in goodbye_indeces:
    print('dlg_id -',dlg, 'line_n -', line, ':', 
          df[(df.dlg_id == dlg) & (df.line_n == line)].text.iloc[0])

dlg_id - 0 line_n - 108 : Всего хорошего до свидания
dlg_id - 1 line_n - 53 : Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего
dlg_id - 1 line_n - 54 : До свидания
dlg_id - 3 line_n - 51 : Угу все хорошо да понедельника тогда всего доброго
dlg_id - 4 line_n - 33 : Во вторник все ну с вами да тогда до вторника до свидания
dlg_id - 5 line_n - 142 : Ну до свидания хорошего вечера


### Менеджер представил себя

In [9]:
def extrant_manager_introduce_himself(dataframe)->list[tuple()]:
    intr_hmslf_str = re.compile(' это | зовут | звать ')
    mngr_intr_hmslf_indeces = []
    for idx, text in enumerate(dataframe.text):
        if dataframe.line_n[idx] > 3:
            continue
        if dataframe.role[idx] == 'client':
            continue
        if intr_hmslf_str.findall(text.lower()):
            mngr_intr_hmslf_indeces.append((dataframe.dlg_id[idx], dataframe.line_n[idx]))
    return mngr_intr_hmslf_indeces

In [10]:
# test manager introduce himself
mngr_intr_hmslf_indeces = extrant_manager_introduce_himself(df)
for dlg, line in mngr_intr_hmslf_indeces:
    print('dlg_id -',dlg, 'line_n -', line, ':', 
          df[(df.dlg_id == dlg) & (df.line_n == line)].text.iloc[0])

dlg_id - 0 line_n - 3 : Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
dlg_id - 1 line_n - 2 : Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается
dlg_id - 2 line_n - 3 : Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там
dlg_id - 3 line_n - 2 : Добрый меня максим зовут компания китобизнес удобно говорить
dlg_id - 5 line_n - 1 : Да это анастасия


### Имя менеджера

In [11]:
def extract_manager_name(dataframe, mngr_intr_hmslf_indeces)->dict:
    # возвращает словарь {диалог: имя менеджера}
    manager_name = {}
    # probability score threshold
    prob_thresh = 0.4
    morph = pymorphy2.MorphAnalyzer()
    for dlg, line in mngr_intr_hmslf_indeces:
        text = dataframe[(dataframe.dlg_id == dlg) & (dataframe.line_n == line)].text.iloc[0]
        for word in nltk.word_tokenize(text):
            for p in morph.parse(word):
                if 'Name' in p.tag and p.score >= prob_thresh:
                    manager_name[dlg] = word.capitalize()
    return manager_name

In [12]:
# test
d = extract_manager_name(df, mngr_intr_hmslf_indeces)
print(d)

{0: 'Ангелина', 1: 'Ангелина', 2: 'Ангелина', 3: 'Максим', 5: 'Анастасия'}


### Название компании

In [13]:
def extract_name_company(dataframe, mngr_intr_hmslf_indeces)->dict:
    company_name_str = re.compile('компания\s\w*\s\w*')
    company_name = {}
    for dlg, line in mngr_intr_hmslf_indeces:
        text = dataframe[(dataframe.dlg_id == dlg) & (dataframe.line_n == line)].text.iloc[0]
        result = company_name_str.findall(text.lower())
        if result:
            company_name[dlg] = result
    return company_name

In [14]:
print(extract_name_company(df, mngr_intr_hmslf_indeces))

{0: ['компания диджитал бизнес'], 1: ['компания диджитал бизнес'], 2: ['компания диджитал бизнес'], 3: ['компания китобизнес удобно']}


### Проверка что менеджер поздоровался и попрощался в диалоге

In [15]:
def is_polite_manager(dataframe):
    # Возвращает словарь {номер диалога: True если поприветсвовал и попрощался, иначе False}
    dlg1 = set()
    dlg2 = set()
    result = {}
    for dlg, _ in extract_greeting(dataframe):
        dlg1.add(dlg)
    for dlg, _ in extract_goodbye(dataframe):
        dlg2.add(dlg)
    for el in dlg1:
        if el in dlg2:
            result[el] = True
        else: 
            result[el] = False
    return result

In [16]:
is_polite_manager(df)

{0: True, 1: True, 2: False, 3: True}